In [ ]:
import urllib.request, urllib.parse, urllib.error
import ssl
import json
import twurl
import sqlite3
# ignore twitter certificate
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE
# send request to sql and received
TWITTER_URL = 'https://api.twitter.com/1.1/friends/list.json'
conn = sqlite3.connect('friends.sqlite')
cur = conn.cursor()
# refresh the existing table
cur.execute('''CREATE TABLE IF NOT EXISTS People 
            (id INTEGER PRIMARY KEY, name TEXT UNIQUE, retrieved INTEGER)''')
cur.execute('''CREATE TABLE IF NOT EXISTS Follows
            (from_id INTEGER, to_id INTEGER, UNIQUE(from_id, to_id))''')
# find id in table people
while True:
    acct = input('Enter a Twitter account, or quit ')
    if acct == 'quit': break
    if len(acct) < 1:
        cur.execute('SELECT (id,name) FROM People WHERE retrieved =0 LIMIT 1')
        try:
            (id, acct) = cur.fetchone()[0]
        except:
            print('No unretrieved Twitter accounts found')
            continue
    else: 
        cur.execute('SELECT id FROM People WHERE name = ? LIMIT 1',
                   (acct, ))
        try:
            id = cur.fetchone()[0]
        except:
            cur.execute('''INSERT OR IGNORE INTO People
                        (name, retrieved) VALUES (?, 0)''',
                        (acct, ))
            conn.commit()
            if cur.rowcount != 1:
                print('Error inserting account:', acct)
                continue
            id = cur.lastrowid
# find id in friends 
    url = twurl.augment(TWITTER_URL, {'screen_name': acct, 'count': 100})
    print('Retrieving account', acct)
    try:
        connection = urllib.request.urlopen(url, context=ctx)
    except Exception as err:
        print('Failed to retrieve', err)
    data = connection.read().decode()
    headers = dict(connection.getheaders())
    
    print('Remaining', headers['x-rate-limit-remaining'])
    
    try:
        js = json.loads(data)
    except:
        print('Unable to parse json')
        print(data)
        break
        
    if 'users' not in js:
        print('Incorrect JSON received')
        print(json.dumps(js, indent=4))
    
    cur.execute('UPDATE People SET retrieved = 1 WHERE name = ?',
               (acct, ))
    countold = 0
    countnew = 0
    for u in js['users']:
        friend = u['screen_name']
        print(friend)
        cur.execute('SELECT id FROM People WHERE name = ? LIMIT 1',
                   (friend, ))
        try:
            friend_id = cur.fetchone()[0]
            countold = countold + 1
        except:
            cur.execute('''INSERT OR IGNORE People (name, retrieved)
                VALUES (?, 0)''', (friend, ))
            conn.commit()
            if cur.rowcount != 1:
                print('Error inserting account:', friend)
                continue
            friend_id = cur.lastrowid
            countnew = countnew + 1
        cur.execute('''INSERT OR IGNORE INTO Follows (from_id, to_id)
                    VALUES (?, ?)''', (id, friend_id))
    print('New accounts=', countnew, 'revisited=', countold)
    print('Remaining', headers['x-rate-limit-remaining'])
    conn.commit()
cur.close() 
        

Enter a Twitter account, or quit drchuck
Retrieving account drchuck
Remaining 10
Una
cherybrown
addyosmani
jenn543
MozDevNet
djangogirls
olasitarska
CarnesBeau
shastri_sunil
ossia
raghavgupta72
manishavasdev
ad1tya5
Bibinsr
sairajreddy10
moorepants
Wilma_Hodges
Lingamarla
cshirky
lingamarlas
visionaid1
LibreTexts
OSCELOT_Inc
SakaiCar
trtsmb
djangoproject
pythonanywhere
PerfForensics
dog_feelings
LeboGuitar
24HRSoflemonsau
MrTrashWheel
eduappcenter
IFloFreely
shurrey
OnlineCrsLady
miguelamigot
S_Phillipson
24hoursoflemons
kosburn0408
DEVlANT
jrochelle
ElizabethCorish
samleepan
SusanCampher
vajkat
nurigocay
ITintheD
J0hnnyXm4s
RegexTip
SimonRoofingMI
Cmdr_Hadfield
GoogleForEdu
tsugiproject
TheLearnXP
rkniemer
DetroitRugby
4tvirtualcon
jfrwhyte
IronStache
phpdet
FrankLaszlo
victoriastodden
USQRA
vmbrasseur
evilbmcats
nanmuntin1
susudancer
openmatter
KarutaOSP
SpaceX
Bourdain
rmengod
HigherEdSurge
DanielTechnical
Penguicon
_mike_collins
schoolcity
andyfmiller
jennifervetter
Bob_Wachter
kat